In [1]:
import numpy as np
import time
import cv2
import os
import subprocess

In [3]:
LABELS = open("coco.names").read().strip().split("\n")


print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet("yolov3.cfg", "yolov3.weights")


ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

[INFO] loading YOLO from disk...


In [4]:
import imageio
from PIL import Image

In [11]:
def take_photo(imagepath):
    frame = imageio.imread(imagepath)
# print(frame.shape)
    (H, W) = frame.shape[:2]
#     print(H, W)
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
    swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)


    boxes = []
    confidences = []
    classIDs = []
    centers = []

# loop over each of the layer outputs
    for output in layerOutputs:
# loop over each of the detections


        for detection in output:


            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

       
            if confidence > 0.5:

                        box = detection[0:4] * np.array([W, H, W, H])
                        (centerX, centerY, width, height) = box.astype("int")


                        x = int(centerX - (width / 2))
                        y = int(centerY - (height / 2))


                        boxes.append([x, y, int(width), int(height)])
                        confidences.append(float(confidence))
                        classIDs.append(classID)
                        centers.append((centerX, centerY))


        idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)
        texts = {}
        l = 0

#         print(confidences)
        if len(idxs) > 0:

            for i in idxs.flatten():

                
                centerX, centerY = centers[i][0], centers[i][1]
                if centerX <= W/3:
                    W_pos = "left "
                elif centerX <= (W/3 * 2):
                    W_pos = "center "
                else:
                    W_pos = "right "

                if centerY <= H/3:
                    H_pos = "top" 
                elif centerY <= (H/3 * 2):
                    H_pos = "mid "
                else:
                    H_pos = "bottom "

#                 texts.append('H_pos + W_pos + {0} + {1} + {2} + {3} + {4}'.format(LABELS[classIDs[i]],confidences[i], centerX, centerY, boxes[i]))
                res = {'height': H_pos, 'width': W_pos, 'label': LABELS[classIDs[i]], 'confidence_score': confidences[i], 'centerX': centerX, 'centerY': centerY, 'box_parameter': boxes[i]}
                texts[l] = res
                l = l + 1

    return (texts)

In [12]:
take_photo("test.jpg")

{0: {'height': 'mid ',
  'width': 'center ',
  'label': 'person',
  'confidence_score': 0.9993494749069214,
  'centerX': 704,
  'centerY': 676,
  'box_parameter': [241, 198, 925, 956]},
 1: {'height': 'top',
  'width': 'left ',
  'label': 'bird',
  'confidence_score': 0.923171877861023,
  'centerX': 388,
  'centerY': 388,
  'box_parameter': [308, 329, 160, 117]},
 2: {'height': 'top',
  'width': 'center ',
  'label': 'bird',
  'confidence_score': 0.9153757691383362,
  'centerX': 1074,
  'centerY': 152,
  'box_parameter': [1032, 113, 84, 77]},
 3: {'height': 'top',
  'width': 'center ',
  'label': 'bird',
  'confidence_score': 0.8959580659866333,
  'centerX': 787,
  'centerY': 65,
  'box_parameter': [750, 48, 74, 33]},
 4: {'height': 'mid ',
  'width': 'right ',
  'label': 'bird',
  'confidence_score': 0.8692618608474731,
  'centerX': 1386,
  'centerY': 478,
  'box_parameter': [1338, 449, 95, 57]},
 5: {'height': 'top',
  'width': 'right ',
  'label': 'bird',
  'confidence_score': 0.805